# 02 Exploratory Data Analysis

Notebook goal: initial Exploratory Data Analysis to understand columns (features), data cleanliness and basic (inter)correlations:

1. Check data types of columns and re-cast if necessary
2. Check validity of LENGTH_OF_STAY column
3. Check ordering of dataset
4. Check levels of missing data
4. Check levels of duplication of dataset
5. Basic correlation to look at relationships between columns
6. Generate a pandas profile and explore distributions of data

Note that EDA is a cyclical process, and many explorations of the data take place after cleaning.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from pandas_profiling import ProfileReport

%matplotlib inline

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## 1. Load data

Data processed in previous notebook

In [ ]:
# Load into pandas
original_data_df = pd.read_parquet("../../data/raw/original-data.parquet")

## Explore Data types

Check the range of data types in the dataset manually:

In [ ]:
original_data_df.dtypes

There are two derived fields, `arrival_day_of_week` and `arrival_month_name` - how have they been derived?

In [ ]:
original_data_df.arrival_day_of_week.unique()

In [ ]:
original_data_df.arrival_month_name.unique()

## Data consistency checks

Does the LENGTH_OF_STAY match start/end dates?

In [ ]:
# check data types before conducting maths
original_data_df[
    [
        "DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL",
        "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL",
    ]
].dtypes

In [ ]:
# cast dates to datetime
datetime_df = original_data_df.copy()
datetime_df.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL = pd.to_datetime(
    datetime_df.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL, format="%Y-%m-%d %H:%M:%S.%f"
)
datetime_df.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL = pd.to_datetime(
    datetime_df.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
# Discharge is whole day, admission is datetime
datetime_df[
    [
        "DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL",
        "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL",
    ]
].sample(10)

In [ ]:
# calculate derived LoS
# round up to whole days
datetime_df["DER_los"] = (
    datetime_df["DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL"]
    - datetime_df["START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]
).dt.days + 1

In [ ]:
# quick visual inspection - do they match?
datetime_df[["DER_los", "LENGTH_OF_STAY"]].head(10)

In [ ]:
# check that mean difference is ~ 0 days
datetime_df[["DER_los", "LENGTH_OF_STAY"]].diff(axis=1).LENGTH_OF_STAY.mean()

## Data ordering

How is data ordered?

In [ ]:
# not ordered by local patient id
datetime_df.LOCAL_PATIENT_IDENTIFIER.head(10)

# nb. value_counts() shows repeat visits - could this be feature?

In [ ]:
# not ordered by start-date
datetime_df.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL.head(10)

In [ ]:
# not by end-date
datetime_df.DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL.head(10)

In [ ]:
# not by cds
datetime_df.cds_unique_identifier.sample(10)

Re-order data by `START_DATE_TIME_HOSPITAL_PROVIDER_SPELL`:

In [ ]:
datetime_df.sort_values(by="START_DATE_TIME_HOSPITAL_PROVIDER_SPELL", inplace=True)
datetime_df.reset_index(drop=True, inplace=True)

### Explore Missing data

We can generate a heatmap of missing data to quickly visualise the totality of missing data in one image. This will only capture significant areas of missing data, but can be useful to identify sparse columns, rows and blocks of missing data.

The heatmap generated will show missing data in black, and present data in white.

Any patches of black indicate missing data:

In [ ]:
sns.set(rc={"figure.figsize": (15, 8)})
sns.heatmap(datetime_df.isnull(), cbar=False);

We can also summarise the number of missing rows of data in tabular format, which can help identify columns with smaller amounts of missing data:

In [ ]:
datetime_df.isnull().sum()

The population for this project is major, non-elective cases. The `IS_major` field capture major cases, but we want to check how complete it is as it will be used to define the population.

In [ ]:
datetime_df.IS_major.value_counts(dropna=False)

Likewise, we want to check the `elective_or_non_elective` column:

In [ ]:
datetime_df.elective_or_non_elective.value_counts(dropna=False)

## Identify early block of missing data

The first ~100k rows (from visual plot) are missing a significant amount of data - is this due to the introduction of a new system?

1. Which columns are missing?
1. When does the block of missing data end? Does this align with a new clinical system?

Visually, a one column is that is indicative of the missing data is `wait_minutes`. We will use this column as an indicator for the missing data.

In [ ]:
# extract roughly the missing block
missing_block_df = datetime_df.iloc[0:120000]
# which columns are null when wait_minutes is null?
missing_rows_df = missing_block_df[missing_block_df.wait_minutes.isna()]
# calculate proportion of missing rows
missing_fields_df = pd.DataFrame(missing_rows_df.isna().sum().reset_index())
missing_fields_df.columns = ["field", "null_count"]
missing_fields_df["null_prop"] = (
    missing_fields_df["null_count"] / missing_rows_df.shape[0]
)
# select columns that have between 95% and 101% missing rows, to capture similar levels of missing data
missing_fields_df[missing_fields_df.null_prop.between(0.95, 1.01)]

Roughly 39 columns are missing similar levels of data to `wait_minutes` in the first 120,000 rows of data.

These align with the introduction of a new system. 

We will identify which row index this system was introduced, visually then by reviewing the data by hand.

In [ ]:
# Plot an interactive view of single column around where the gap starts
# Use plotly zoom functionality to identify where the missing data ends
fig = px.line(
    datetime_df.iloc[100000:120000],
    x=datetime_df.iloc[100000:120000].index,
    y="wait_minutes",
)
fig.show()

Check on the area of interest by examining values by hand, in a tabular format. We are looking for the start of a continuous presence of the `wait_minutes` feature, bearing in mind the column is semi-sparse.

In [ ]:
datetime_df.iloc[109500:110000].wait_minutes

After inspection by eye (plot) and by hand (table) we define index `109894` as the start of "good quality data", based on the presence of `wait_minutes` data which is indicative of the missing columns at the start of the dataframe.

## Correlation plot

In [ ]:
# Pearson correlation by default:
corr = datetime_df.corr()

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 8))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

In [ ]:
# check for correlation between features
corr[corr != 1.00][corr.abs() > 0.1].abs().unstack().sort_values(ascending=False)

In [ ]:
# check for correlation with LENGTH OF STAY
corr.LENGTH_OF_STAY[corr.LENGTH_OF_STAY.abs().sort_values(ascending=False).index]

## Check duplicate rows

In [ ]:
datetime_df.duplicated().sum()

## Check duplicate columns

In [ ]:
# How is FIRST_START_DATE_TIME_WARD_STAY different to START_DATE_TIME_HOSPITAL_PROVIDER_SPELL?
# Cast FIRST_START_DATE_TIME_WARD_STAY to datetime
datetime_df.FIRST_START_DATE_TIME_WARD_STAY = pd.to_datetime(
    datetime_df.FIRST_START_DATE_TIME_WARD_STAY, format="%Y-%m-%d %H:%M:%S.%f"
)

In [ ]:
# check if FIRST_START_DATE_TIME_WARD_STAY is the same as START_DATE_TIME_HOSPITAL_PROVIDER_SPELL
datetime_df.FIRST_START_DATE_TIME_WARD_STAY.equals(
    datetime_df.START_DATE_TIME_HOSPITAL_PROVIDER_SPELL
)

In [ ]:
# they are different, so work out what the difference is between the columns
datetime_df[
    ["FIRST_START_DATE_TIME_WARD_STAY", "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]
].sample(10).diff(axis=1)

In [ ]:
# there are many NaT values in FIRST_START_DATE_TIME_WARD_STAY which lead to a difference of 0 days
# find out if there are any actual differences in dates
(
    datetime_df[
        ["FIRST_START_DATE_TIME_WARD_STAY", "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL"]
    ]
    .diff(axis=1)
    .START_DATE_TIME_HOSPITAL_PROVIDER_SPELL
    > pd.Timedelta(0)
).sum()

## Pandas profiling

Pandas profiling will (after some time) generate an overall profile of the dataset, including histograms, frequent values and checks such as sparsity and ordinality that can help generate further questions for the data subject matter expert (SME).

In [ ]:
pd.__version__
# note bug with version 1.4.1: https://github.com/ydataai/pandas-profiling/issues/911
# use lower version (e.g. 1.3.5)

In [ ]:
# Dataset large and crashing without minimal=True
profile = ProfileReport(datetime_df, title="Pandas Profiling Report", minimal=True)

In [ ]:
profile